In [ ]:
import numpy as np 
import pandas as pd
import sklearn
import lightgbm as lgb
import matplotlib.pyplot as plt
from tqdm import tqdm
import seaborn as sns
import gc
from collections import defaultdict  
import math  
import Standard_itemcf
import Yu_itemcf_TimeSeries
import Bipartite_network_CF
import userCF
import MyEvaluation
import random
from random import sample

# -*- coding: utf-8 -*-
from sklearn.model_selection import train_test_split
import scipy.sparse as sp
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense,Dropout,Activation,Input,Convolution1D,Conv1D,GlobalMaxPooling1D,MaxPooling1D,Flatten,concatenate,Embedding,GRU,Lambda, LSTM, TimeDistributed
from keras.layers.merge import Concatenate
from keras.models import model_from_json
from keras.preprocessing.text import Tokenizer
from keras.utils.np_utils import to_categorical
from keras.wrappers.scikit_learn import KerasClassifier
from keras.optimizers import RMSprop
from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split
from keras.models import load_model,Model
from keras.layers import Dense, Embedding, LSTM,Flatten,BatchNormalization
from gensim.models import word2vec 
import keras
import time
from keras.optimizers import *
from keras.callbacks import *
pd.set_option('display.max_rows',200)

import copy

In [ ]:
#is_offline = True
now_phase = 9
train_path = '../input/underexpose_train'  
test_path = '../input/underexpose_test' 
output_path='../output/'
#eval = MyEvaluation.MyEvaluation(now_phase=now_phase)

### Use the items click sequence to pretrain the i2v embedding matrix.

In [ ]:
doc = []
tmp_all = pd.DataFrame()
for c in range(6 + 1):  
    click_train = pd.read_csv(train_path + '/underexpose_train_click-{}.csv'.format(c), header=None,  names=['user_id', 'item_id', 'time'])  
    click_test = pd.read_csv(test_path + '/underexpose_test_click-{}.csv'.format(c), header=None,  names=['user_id', 'item_id', 'time'])  
    tmp = click_train.append(click_test,ignore_index=True).sort_values('time')
    tmp_all = tmp_all.append(tmp,ignore_index=True)
    # 提取用户点击序列，并构成文本
tmp_all.drop_duplicates(inplace=True)
tmp_doc = tmp_all.groupby(['user_id'])['item_id'].agg({list}).reset_index()['list'].values.tolist()
if len(doc)==0 : 
    doc =tmp_doc
else:
    doc.extend(tmp_doc)
    

# 导入 Word2Vec
from gensim.models import Word2Vec

# 转为字符串型才能进行训练
for i in range(len(doc)):
    doc[i] = [str(x) for x in doc[i]]
embed_size=128
model = Word2Vec(doc, size=embed_size, window=50, min_count=3, sg=1, hs=0,negative = 15,iter=200, seed=2020)

# 训练结果提取
values = set(tmp_all['item_id'].values)
w2v=[]

for v in values:
    try:
        a = [int(v)]
        a.extend(model[str(v)])
        w2v.append(a)
    except:
        pass

out_df = pd.DataFrame(w2v)
out_df.columns = ['item_id'] + ['item_vec'+str(i) for i in range(embed_size)]

### Generate the positive samples.

In [ ]:
whole_click = pd.DataFrame()  
for c in range(7,now_phase + 1):  
    click_train = pd.read_csv(train_path + '/underexpose_train_click-{}.csv'.format(c), header=None,  names=['user_id', 'item_id', 'time'])  
    click_test = pd.read_csv(test_path + '/underexpose_test_click-{}.csv'.format(c), header=None,  names=['user_id', 'item_id', 'time'])  
    all_click = click_train.append(click_test)  
    whole_click = whole_click.append(all_click)  

whole_click = whole_click.drop_duplicates(subset=['user_id','item_id','time'])
whole_click = whole_click.sort_values('time',ascending=True)   
whole_click = whole_click.reset_index(drop=True)
for col in ['item_id','time']:#,'time_interval_cumsum']:
    whole_click[col]=whole_click[col].astype(str)

whole_item_df = whole_click.groupby('user_id').agg(
                                item_list = pd.NamedAgg(column = 'item_id',aggfunc=(lambda x : list(x))),
                                time_list = pd.NamedAgg(column = 'time',aggfunc=(lambda x : list(x))),
                                ).reset_index()

In [ ]:
positive_df = []
last_n = 5
for i in tqdm(whole_item_df.index):
    user_id = whole_item_df.loc[i,'user_id']
    item_list = whole_item_df.loc[i,'item_list']
    time_list = whole_item_df.loc[i,'time_list']
    if len(item_list)<=1: 
        continue
    if len(item_list)<last_n: 
        for j in range(3,len(item_list)+1):
            positive_df.append([user_id,' '.join(item_list[:j]),time_list[j-1]])
    else:
        for j in range(3,last_n):
            positive_df.append([user_id,' '.join(item_list[:j]),time_list[j-1]])
        for j in range(last_n,len(item_list)+1,1):
            recent_item_list = item_list[j-last_n:j]
            positive_df.append([user_id,' '.join(recent_item_list),time_list[j-1]])    

In [ ]:
train_set_df_p = pd.DataFrame(positive_df,columns=['user_id','recent_items','time'])
train_set_df_p['label'] = 1
train_set_df_p['target_item'] = train_set_df_p['recent_items'].apply(lambda x : x.split(' ')[-1])
train_set_df_p['recent_items'] = train_set_df_p['recent_items'].apply(lambda x : ' '.join(x.split(' ')[:-1]))

In [ ]:
#train_set_df_p = train_set_df_p.sample(frac=0.4)

### Generate the negative samples that 100 times more than positive samples.

In [ ]:
negative_ratio = 100
negative_list = whole_click['item_id'].sample(len(train_set_df_p)*negative_ratio,replace=True).values.tolist()
train_set_df = train_set_df_p[['user_id','time','target_item','label']].copy()
for i in tqdm(range(negative_ratio)):
    tmp_negative_list = negative_list[i*len(train_set_df_p):(i+1)*len(train_set_df_p)]
    tmp_df = train_set_df_p[['user_id','time','target_item','label']].copy()
    tmp_df['target_item']=tmp_negative_list
    tmp_df['label'] = 0
    train_set_df=train_set_df.append(tmp_df,ignore_index=True)

### Load the user's feature.

In [ ]:
user_data = pd.read_csv('../input/underexpose_train/underexpose_user_feat.csv',names=['user_id', 'user_age_level', 'user_gender', 'user_city_level'])
user_data = pd.concat([user_data,pd.get_dummies(user_data['user_gender'])],axis=1)
user_data = user_data.drop_duplicates(subset=['user_id'])
user_data.drop(columns=['user_gender'],inplace=True)

train_set_df_p = train_set_df_p.merge(user_data,on='user_id',how='left')

scaler_user = sklearn.preprocessing.StandardScaler()
scaler_user.fit(train_set_df_p[['user_age_level', 'user_city_level','F', 'M']])
train_set_df_p[['user_age_level', 'user_city_level','F', 'M']] = scaler_user.transform(train_set_df_p[['user_age_level', 'user_city_level','F', 'M']])

train_set_df_p['user_age_level'].fillna(0,inplace=True)
train_set_df_p['user_city_level'].fillna(0,inplace=True)
train_set_df_p['F'].fillna(0,inplace=True)
train_set_df_p['M'].fillna(0,inplace=True)

### Remove the last click in different phases. At last we will use them as reference to validade the model's effect.

In [ ]:
test_query = pd.DataFrame()
for c in range(now_phase + 1):  
    tmp_query = pd.read_csv(test_path + '/underexpose_test_qtime-{}.csv'.format(c), header=None,  names=['user_id', 'query_time'])  
    test_query = test_query.append(tmp_query,ignore_index=True)
    
whole_click=whole_click.sort_values(by =['user_id','time'],ascending=False).reset_index(drop=True)    
whole_click['time']=whole_click['time'].astype('float64')
recent_items_df = pd.DataFrame()
for i in tqdm(test_query.index):
    user = test_query.loc[i,'user_id']
    time = test_query.loc[i,'query_time']
    time_left = str(whole_click.loc[(whole_click['user_id']==user)&(whole_click['time']<time)]['time'].max())
    time_right =  str(whole_click.loc[(whole_click['user_id']==user)&(whole_click['time']>time)]['time'].min())
    recent_items_df =recent_items_df.append([{'user_id':user,'time':time_left,'isval':1},{'user_id':user,'time':time_right,'isval':2}],ignore_index=True)
recent_items_df = recent_items_df[recent_items_df['time']!='nan']
train_set_df = train_set_df.merge(recent_items_df,on=['user_id','time'],how='left')

### Split the data set into train set and valid set.

In [ ]:
test_set_df = train_set_df[train_set_df['isval']==1].reset_index(drop=True)
train_set_df = train_set_df[train_set_df['isval'].isnull()].reset_index(drop=True)

val_data_ratio = 20
random.seed(2020)
sample_ind = sample(range(train_set_df.label.sum()),train_set_df.label.sum()//val_data_ratio)
train_sample_ind = list(set(list(range(train_set_df.label.sum()))) - set(sample_ind))
val_sample_ind = sample_ind[:len(sample_ind)]

val_sample_ind_all = []
for i in range(negative_ratio+1):
    tmp_ind = np.add(val_sample_ind,train_set_df.label.sum()*i)
    val_sample_ind_all.extend(tmp_ind)

train_sample_ind_all = []
for i in range(negative_ratio+1):
    tmp_ind = np.add(train_sample_ind,train_set_df.label.sum()*i)
    train_sample_ind_all.extend(tmp_ind)
    

test_len = test_set_df.shape[0]
train_set_df = train_set_df.append(test_set_df,ignore_index=True)

In [ ]:
whole_item_df['item_list'] = whole_item_df['item_list'].apply(lambda x : ' '.join([str(i) for i in x]))
max_word_len = whole_click.item_id.nunique()
tokenizer = Tokenizer(num_words=max_word_len, split=' ')
tokenizer.fit_on_texts(whole_item_df['item_list'])
recent_items_list = pad_sequences(tokenizer.texts_to_sequences(train_set_df_p['recent_items']))
train_set_df_p['recent_items5'] = recent_items_list.tolist()
train_set_df['target_item'] = pad_sequences(tokenizer.texts_to_sequences(train_set_df['target_item'])).tolist()

In [ ]:
out_df = out_df.set_index('item_id')
max_feaures = max_word_len
embedding_matrix = np.zeros((max_feaures+1, embed_size))
word_index_list = list(tokenizer.word_index.keys())
for ind in range(len(word_index_list)):
    word_ = word_index_list[ind]
    word = int(word_)
    if word not in out_df.index:
        continue
    embedding_matrix[tokenizer.word_index[word_]] = out_df.loc[word].values

In [ ]:
test_set_df = train_set_df[-test_len:]
val_set_df = train_set_df.loc[val_sample_ind_all]
train_set_df = train_set_df.loc[train_sample_ind_all]

### Drop the duplicate items in different set.

In [ ]:
train_set_df['target']=train_set_df['target_item'].apply(lambda x :x[0])
train_set_df.drop_duplicates(subset=['user_id','time','target'],keep='first',inplace=True)
train_set_df.drop(columns=['target','isval'],inplace = True)

val_set_df['target']=val_set_df['target_item'].apply(lambda x :x[0])
val_set_df.drop_duplicates(subset=['user_id','time','target'],keep='first',inplace=True)
val_set_df.drop(columns=['target','isval'],inplace = True)

test_set_df['target']=test_set_df['target_item'].apply(lambda x :x[0])
test_set_df.drop_duplicates(subset=['user_id','time','target'],keep='first',inplace=True)
test_set_df.drop(columns=['target','isval'],inplace = True)

train_set_df = train_set_df.reset_index(drop=True)
val_set_df = val_set_df.reset_index(drop=True)
test_set_df = test_set_df.reset_index(drop=True)

train_set_df_p.drop(columns=['target_item','label'],inplace=True,errors='ignore')

In [ ]:
class data_generator:
    def __init__(self, data, batch_size=128):
        self.data = data
        self.batch_size = batch_size
        self.steps = len(self.data) // self.batch_size
        if len(self.data) % self.batch_size != 0:
            self.steps += 1
    def __len__(self):
        return self.steps

    def __iter__(self):
        global train_set_df_p
        while True:
            train_set = self.data
            idxs = list(range(len(self.data)))
            np.random.shuffle(idxs)
            index_list = []
            X1,  X3 ,Y = [], [], []
            for c, i in enumerate(idxs):
                index_list.append(i)
                if len(index_list) == self.batch_size or i == idxs[-1]:
                    tmp_df = train_set.loc[index_list]
                    tmp_df = tmp_df.merge(train_set_df_p,on=['user_id','time'])
                    tmp_df['recent_items5'] = tmp_df['recent_items5']+tmp_df['target_item']
                    X1 = np.array(tmp_df['recent_items5'].values.tolist())
                    X3 = np.array(tmp_df[['user_age_level', 'user_city_level','F', 'M']].values.tolist())
                    Y = np.array(tmp_df['label'].values.tolist())
                    yield [X1,X3], Y
                    tmp_df = pd.DataFrame()
                    X1, X3 ,Y = [],  [] ,[]
                    index_list = []

### Build the LSTM sort model

In [ ]:
model = Sequential()
seq = Input(shape=[last_n])
emb = Embedding(
           len(embedding_matrix),       #表示文本数据中词汇的取值可能数,从语料库之中保留多少个单词。 因为Keras需要预留一个全零层， 所以+1
           embed_size,              # 嵌入单词的向量空间的大小。它为每个单词定义了这个层的输出向量的大小
           weights=[embedding_matrix],   #构建一个[num_words, EMBEDDING_DIM]的矩阵,然后遍历word_index，将word在W2V模型之中对应vector复制                                        过来。换个方式说：embedding_matrix 是原始W2V的子集，排列顺序按照Tokenizer在fit之后的词顺序。作为权                                      重喂给Embedding Layer
           input_length=last_n,        # 输入序列的长度，也就是一次输入带有的词汇个数
           trainable=False            # 我们设置 trainable = False，代表词向量不作为参数进行更新
           )(seq)
lstm_1 = LSTM(embed_size, input_shape=(last_n, embed_size),return_sequences=True)(emb)
lstm_2 = LSTM(embed_size)(lstm_1)
input_2 = Input(shape=[4,])
merge = concatenate([lstm_2,input_2])
mlp = Dense(units=100,activation='relu')(merge)
mlp = Dropout(0.3)(mlp)
mlp=  BatchNormalization()(mlp)
output = Dense(units=1,activation='sigmoid')(mlp)

model = Model([seq,input_2],output)

model.compile( optimizer=keras.optimizers.RMSprop(1e-3),loss=keras.losses.BinaryCrossentropy(),metrics=[keras.metrics.AUC()])

filepath = "sort_lstm_.h5" 
monitor_name = 'val_auc_1'
checkpoint = ModelCheckpoint(
    filepath, monitor=monitor_name, verbose=1, save_best_only=True, mode='max')
reduce_lr2 = ReduceLROnPlateau(
    monitor=monitor_name, factor=0.8, patience=1, min_lr=0.0001, verbose=1)
earlystopping2 = EarlyStopping(
    monitor=monitor_name, min_delta=0.0001, patience=2, verbose=1, mode='max')
callbacks2 = [checkpoint, earlystopping2 ,reduce_lr2]
train_D = data_generator(train_set_df,batch_size=8192)
valid_D = data_generator(val_set_df,batch_size=8192)
model.fit_generator(train_D.__iter__(),validation_data=valid_D.__iter__(),validation_steps=len(valid_D),steps_per_epoch=len(train_D),epochs=50, shuffle=True,callbacks=callbacks2)

In [ ]:
test_D = data_generator(test_set_df.reset_index(drop=True),batch_size=8192)
model.evaluate_generator(test_D.__iter__(),steps=len(test_D))
model = load_model('./sort_lstm_.h5',compile=False)
model.compile( optimizer=keras.optimizers.RMSprop(1e-3),loss=keras.losses.BinaryCrossentropy(),metrics=[keras.metrics.AUC()])


### Predict the recall items 

In [ ]:
def get_feature(train_set_df , query_df,whole_click,isval=False):
    recent_items_df = pd.DataFrame()
    last_n=5
    for i in tqdm(query_df.index):
        user = query_df.loc[i,'user_id']
        time = query_df.loc[i,'query_time']
        phase = query_df.loc[i,'phase']
        recent_items = ' '.join([str(i) for i in  whole_click.loc[(whole_click['user_id']==user)&(whole_click['time']<time)&(whole_click['phase']==phase)]['item_id'][:last_n-1].values.tolist()[::-1]])
        recent_items_df = recent_items_df.append({'user_id':user,'query_time':time,'recent_items_5':recent_items,'phase':phase},ignore_index=True)
    train_set_df = train_set_df.merge(recent_items_df,on=['user_id','query_time','phase'])
    train_set_df['recent_items_5'] = train_set_df['recent_items_5']+' '+ train_set_df['item_id'].astype(str)
#     tokenizer = joblib.load('./tokenizer.m')
#     scaler_user = joblib.load('./user_info_std_scalar.m')
    user_data = pd.read_csv('../input/underexpose_train/underexpose_user_feat.csv',names=['user_id', 'user_age_level', 'user_gender', 'user_city_level'])
    user_data = pd.concat([user_data,pd.get_dummies(user_data['user_gender'])],axis=1)
    user_data = user_data.drop_duplicates(subset=['user_id'])
    user_data.drop(columns=['user_gender'],inplace=True)
    train_set_df = train_set_df.merge(user_data,on='user_id',how='left')
    train_set_df[['user_age_level', 'user_city_level','F', 'M']] = scaler_user.transform(train_set_df[['user_age_level', 'user_city_level','F', 'M']])
    train_set_df['user_age_level'].fillna(0,inplace=True)
    train_set_df['user_city_level'].fillna(0,inplace=True)
    train_set_df['F'].fillna(0,inplace=True)
    train_set_df['M'].fillna(0,inplace=True)
    X = tokenizer.texts_to_sequences(train_set_df['recent_items_5'])
    X = pad_sequences(X)
    if isval:
        return X,train_set_df[['user_age_level', 'user_city_level','F', 'M']],train_set_df['label']
        #return train_set_df
    else:
        return X,train_set_df[['user_age_level', 'user_city_level','F', 'M']]

In [ ]:
final_recall = pd.read_pickle('../user_data/data/recall.pkl')
final_recall = final_recall[['user_id', 'phase', 'query_time', 'item_id', 'label']]
train_set_df = final_recall[~final_recall['label'].isnull()].reset_index(drop=True)
test_set_df = final_recall[final_recall['label'].isnull()].reset_index(drop=True)
query_df = train_set_df[train_set_df['label']==1]

now_phase = 9
whole_click = pd.DataFrame()  
for c in range(7,now_phase + 1):  
    click_train = pd.read_csv(train_path + '/underexpose_train_click-{}.csv'.format(c), header=None,  names=['user_id', 'item_id', 'time'])  
    click_test = pd.read_csv(test_path + '/underexpose_test_click-{}.csv'.format(c), header=None,  names=['user_id', 'item_id', 'time'])  
    all_click = click_train.append(click_test)  
    all_click['phase']=c
    whole_click = whole_click.append(all_click)  
whole_click = whole_click.drop_duplicates(subset=['user_id','item_id','time','phase'])
whole_click = whole_click.sort_values(by =['user_id','time'],ascending=False).reset_index(drop=True)    


In [ ]:
X_1,X_2,y = get_feature(train_set_df,query_df,whole_click,isval=True)
train_set_df['pred'] = lstm_model.predict([X_1,X_2],batch_size=4096)

In [ ]:
test_query = test_set_df.drop_duplicates(['user_id','phase','query_time'])
test_x_1,test_x_2 =get_feature(whole_click=whole_click,train_set_df=test_set_df,query_df=test_query,isval=False,)
test_set_df['pred'] = lstm_model.predict([test_x_1,test_x_2 ],batch_size=4096)

In [ ]:
data_all = train_set_df.append(test_set_df,ignore_index=True)
data_all.to_pickle('../user_data/data/nn/nn2.pkl')